In [1]:
import time

In [10]:
bitMaps = []

for i in range(8):
    tempBitMap = []
    for j in range(8):
        dirX, dirY = [0, -1]
        tempX, tempY = i+dirX, j+dirY
        bits = 0
        while tempX <= 7 and tempY <= 7 and tempX >= 0 and tempY >=0:
            bits |= 1<<(63-tempX*8-tempY)
            tempX += dirX
            tempY += dirY
        tempBitMap.append(bits)
    bitMaps.append(tempBitMap)

print(bitMaps)

[[0, 9223372036854775808, 13835058055282163712, 16140901064495857664, 17293822569102704640, 17870283321406128128, 18158513697557839872, 18302628885633695744], [0, 36028797018963968, 54043195528445952, 63050394783186944, 67553994410557440, 69805794224242688, 70931694131085312, 71494644084506624], [0, 140737488355328, 211106232532992, 246290604621824, 263882790666240, 272678883688448, 277076930199552, 279275953455104], [0, 549755813888, 824633720832, 962072674304, 1030792151040, 1065151889408, 1082331758592, 1090921693184], [0, 2147483648, 3221225472, 3758096384, 4026531840, 4160749568, 4227858432, 4261412864], [0, 8388608, 12582912, 14680064, 15728640, 16252928, 16515072, 16646144], [0, 32768, 49152, 57344, 61440, 63488, 64512, 65024], [0, 128, 192, 224, 240, 248, 252, 254]]


In [11]:
for x, row in enumerate(bitMaps):
    for y, bits in enumerate(row):
        for i in range(8):
            for j in range(8):
                if x==i and y==j:
                    print('2', end=' ')
                else:
                    if bits & (1 << (63 - 8*i-j)):
                        print('1', end=' ')
                    else:
                        print('0', end=' ')
            print()
        print()
        time.sleep(1)

        

2 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 

1 2 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 

1 1 2 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 

1 1 1 2 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 

1 1 1 1 2 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 

1 1 1 1 1 2 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 



KeyboardInterrupt: 